In [1]:
# Evaluation of enhanced RAG pipeline - results in results folder.
#Setting up
import sys
sys.path.insert(0, '../src')

from enhanced_rag import EnhancedRAG
from utils import calculate_f1_score, exact_match
import pandas as pd
from tqdm import tqdm

print("Initializing Enhanced RAG...")
enhanced_rag = EnhancedRAG(embedding_model_name="all-mpnet-base-v2")
documents, qa_pairs = enhanced_rag.load_dataset()
enhanced_rag.create_embeddings(batch_size=32)
enhanced_rag.build_vector_db()
enhanced_rag.load_generator()

print("\nEnhanced RAG ready")
print("Enhancements: Re-ranking + Query Rewriting")

2025-10-03 17:49:04,133 - naive_rag - INFO - Initializing RAG with embedding model: all-mpnet-base-v2
2025-10-03 17:49:04,133 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-10-03 17:49:04,133 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2


Utils module loaded successfully!
Initializing Enhanced RAG...


2025-10-03 17:49:05,909 - naive_rag - INFO - RAG system initialized successfully
2025-10-03 17:49:05,909 - enhanced_rag - INFO - Loading cross-encoder for re-ranking...


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

C:\Users\vashi\anaconda3\envs\rag-assignment\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vashi\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fallin

...

README.md: 0.00B [00:00, ?B/s]

2025-10-03 17:49:09,769 - sentence_transformers.cross_encoder.CrossEncoder - INFO - Use pytorch device: cpu
2025-10-03 17:49:09,906 - enhanced_rag - INFO - Cross-encoder loaded
2025-10-03 17:49:09,906 - naive_rag - INFO - Loading RAG Mini Wikipedia dataset...
2025-10-03 17:49:09,906 - naive_rag - INFO - Loading text corpus...
2025-10-03 17:49:12,218 - naive_rag - INFO - Loading Q&A pairs...
2025-10-03 17:49:13,471 - naive_rag - INFO - Loaded 3200 documents and 918 Q&A pairs
2025-10-03 17:49:13,471 - naive_rag - INFO - Creating embeddings for 3200 documents...


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

2025-10-03 17:56:29,554 - naive_rag - INFO - Created embeddings with shape: (3200, 768)
2025-10-03 17:56:29,555 - naive_rag - INFO - Building FAISS vector database...
2025-10-03 17:56:29,557 - naive_rag - INFO - FAISS index built with 3200 vectors
2025-10-03 17:56:29,557 - naive_rag - INFO - Loading generation model: google/flan-t5-base
2025-10-03 17:56:30,638 - naive_rag - INFO - Generation model loaded successfully



✅ Enhanced RAG ready
Enhancements: Re-ranking + Query Rewriting


In [3]:
#Evaluating 50 questions for quick evaluations
test_size = 50
test_questions = [qa_pairs[i] for i in range(test_size)]

# Test different enhancement combinations
configs = [
	{'name': 'Baseline (no enhancements)', 'rerank': False, 'rewrite': False},
	{'name': 'Re-ranking only', 'rerank': True, 'rewrite': False},
	{'name': 'Query rewriting only', 'rerank': False, 'rewrite': True},
	{'name': 'Both enhancements', 'rerank': True, 'rewrite': True}
]

print(f"Testing {len(configs)} configurations on {test_size} questions")

Testing 4 configurations on 50 questions


In [7]:
#Evaluating
results = []

for config in configs:
	print(f"\n")
	print(f"Testing: {config['name']}")
	print("\n")
	
	scores = []
	for qa in tqdm(test_questions, desc=config['name']):
		try:
			result = enhanced_rag.query(
				qa['question'],
				top_k=3,
				prompt_strategy="basic",
				use_reranking=config['rerank'],
				use_rewriting=config['rewrite']
			)
			
			f1 = calculate_f1_score(result['answer'], qa['answer'])
			em = exact_match(result['answer'], qa['answer'])
			scores.append({'f1': f1, 'em': em})
		except:
			scores.append({'f1': 0.0, 'em': False})
	
	avg_f1 = sum(s['f1'] for s in scores) / len(scores)
	avg_em = sum(s['em'] for s in scores) / len(scores)
	
	results.append({
		'Configuration': config['name'],
		'F1 Score': f"{avg_f1:.3f}",
		'Exact Match %': f"{avg_em*100:.1f}%",
		'Reranking': config['rerank'],
		'Query Rewriting': config['rewrite']
	})
	
	print(f"F1: {avg_f1:.3f}, EM: {avg_em*100:.1f}%")

# Display results
df = pd.DataFrame(results)
print("\n")
print("Enhanced RAG Eval Results")
print("\n")
print(df.to_string(index=False))

# Save
df.to_csv('../results/05_enhancement_eval.csv', index=False)
#Evaluation saved to ../results
print("Evaluation saved to ..\results as 05_enhancement_eval")



Testing: Baseline (no enhancements)




Baseline (no enhancements):   0%|                                                               | 0/50 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Baseline (no enhancements):   2%|█                                                      | 1/50 [00:04<03:23,  4.16s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Both enhancements: 100%|███████████████████████████████████████████████████████████████| 50/50 [02:56<00:00,  3.54s/it]

F1: 0.615, EM: 56.0%


Enhanced RAG Eval Results


             Configuration F1 Score Exact Match %  Reranking  Query Rewriting
Baseline (no enhancements)    0.556         50.0%      False            False
           Re-ranking only    0.615         56.0%       True            False
      Query rewriting only    0.576         52.0%      False             True
         Both enhancements    0.615         56.0%       True             True
esults as 05_enhancement_eval
